In [1]:
import cv2
import numpy as np
import heapq

image = cv2.imread('birdseye_view_binary.png')

red_mask = (image[:, :, 2] > 200) & (image[:, :, 1] < 100) & (image[:, :, 0] < 100)

binary_map = np.ones_like(red_mask, dtype=int)
binary_map[~red_mask] = 0

min_distance = 5
start = (307, 321)
end = (746, 564)
# end = (643, 770)

def min_distance_to_red(x, y, red_mask):
    red_coords = np.argwhere(red_mask)
    return np.min(np.sqrt((red_coords[:, 0] - x) ** 2 + (red_coords[:, 1] - y) ** 2))

def a_star(start, end, grid, red_mask):
    open_list = []
    heapq.heappush(open_list, (0 + heuristic(start, end), 0, start))  
    came_from = {}
    g_cost = {start: 0}
    while open_list:
        _, current_g, current = heapq.heappop(open_list)
        if current == end:
            return reconstruct_path(came_from, current)
        
        for neighbor in get_neighbors(current):
            if grid[neighbor] == 1:  
                continue
            
            #if min_distance_to_red(neighbor[0], neighbor[1], red_mask) < min_distance:
            #    continue
            
            tentative_g = current_g + (1.414 if neighbor[0] != current[0] and neighbor[1] != current[1] else 1)
            if neighbor not in g_cost or tentative_g < g_cost[neighbor]:
                g_cost[neighbor] = tentative_g
                f_cost = tentative_g + heuristic(neighbor, end)
                heapq.heappush(open_list, (f_cost, tentative_g, neighbor))
                came_from[neighbor] = current
    return None  

def heuristic(a, b):
    return np.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)

def get_neighbors(current):
    x, y = current
    neighbors = [(x + 1, y), (x - 1, y), (x, y + 1), (x, y - 1), 
                 (x + 1, y + 1), (x + 1, y - 1), (x - 1, y + 1), (x - 1, y - 1)]
    return [n for n in neighbors if 0 <= n[0] < image.shape[0] and 0 <= n[1] < image.shape[1]]

def reconstruct_path(came_from, current):
    path = []
    while current in came_from:
        path.append(current)
        current = came_from[current]
    path.append(current)
    return path[::-1]  

path = a_star(start, end, binary_map, red_mask)

if path:
    for (x, y) in path:
        image[x, y] = [0, 255, 0]  

cv2.imwrite('optimal_path_remote_to_bed.png', image)
# cv2.imwrite('optimal_path_remote_to_sink.png', image)

True